# BC 2
## Single tiny network
## Input = whole image.

In [1]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np
from os import listdir
from os.path import join
import cv2
import pandas as pd
import os
import random as rn
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
SEED = 321
SCALE = 32
rn.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [3]:
data = "../input/flowers-recognition/flowers/"
folders = os.listdir(data)

In [4]:
image_names = []
train_labels = []
train_images = []

size = 32,32

for folder in folders:
    for file in os.listdir(os.path.join(data,folder)):
        if file.endswith("jpg"):
            image_names.append(os.path.join(data,folder,file))
            train_labels.append(folder)
            img = cv2.imread(os.path.join(data,folder,file))
            im = cv2.resize(img,size)
            train_images.append(im)
        else:
            continue

In [5]:
train = np.array(train_images)

train.shape

(4323, 32, 32, 3)

In [6]:
train = train.astype('float32') / 255.0

In [7]:
label_dummies = pd.get_dummies(train_labels)

labels =  label_dummies.values.argmax(1)

In [8]:
pd.unique(train_labels)

array(['dandelion', 'daisy', 'sunflower', 'tulip', 'rose'], dtype=object)

In [9]:
pd.unique(labels)

array([1, 0, 3, 4, 2])

In [10]:
union_list = list(zip(train, labels))
rn.shuffle(union_list)
train,labels = zip(*union_list)


train = np.array(train)
labels = np.array(labels)

In [11]:
# Train, Val, Test split = 0.8, 0.1, 0.1 of the dataset
X_train,X_val,y_train,y_val = train_test_split(train,labels, test_size = 0.2)
X_val,X_test,y_val,y_test = train_test_split(X_val,y_val, test_size = 0.5)

In [12]:
#one-hot encoding
y_train_one_hot = to_categorical(y_train)
y_val_one_hot = to_categorical(y_val)
y_test_one_hot = to_categorical(y_test)

In [13]:
#transform to vector
X_train = X_train.reshape((-1, SCALE*SCALE*3))
X_val = X_val.reshape((-1, SCALE*SCALE*3))
X_test = X_test.reshape((-1, SCALE*SCALE*3))

In [14]:
#build ANN model
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=SCALE*SCALE*3))
model.add(Dense(64, activation='relu'))
model.add(Dense(5, activation = 'softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [15]:
model.fit(X_train,y_train_one_hot, epochs=5,batch_size=128,validation_data =(X_val,y_val_one_hot))

Epoch 1/5
28/28 [==============================] - 2s 22ms/step - loss: 1.5598 - accuracy: 0.2922 - val_loss: 1.7728 - val_accuracy: 0.3125
Epoch 2/5
28/28 [==============================] - 0s 5ms/step - loss: 1.5248 - accuracy: 0.3702 - val_loss: 1.3351 - val_accuracy: 0.4028
Epoch 3/5
28/28 [==============================] - 0s 5ms/step - loss: 1.2808 - accuracy: 0.4415 - val_loss: 1.2753 - val_accuracy: 0.4468
Epoch 4/5
28/28 [==============================] - 0s 4ms/step - loss: 1.2360 - accuracy: 0.4643 - val_loss: 1.2457 - val_accuracy: 0.4606
Epoch 5/5
28/28 [==============================] - 0s 5ms/step - loss: 1.2370 - accuracy: 0.4709 - val_loss: 1.2987 - val_accuracy: 0.4167


## Model Evaluation

In [16]:
y_pred = np.argmax(model.predict(X_test), axis=1)
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.3972286374133949